In [79]:
from sys import path
path.insert(0,'..')

from news_api import db_interactions, config

In [72]:
import psycopg
from psycopg import sql

import pandas as pd

In [95]:
def create_news_table() -> None:
    create_table_command = """
    CREATE TABLE IF NOT EXISTS news (
        country VARCHAR(255),
        source_website VARCHAR(255),
        author VARCHAR(255),
        title VARCHAR(255),
        published_at TIMESTAMP
    );
    """
    with psycopg.connect(**config.POSTGRES_CONFIG) as connection:
        cursor = connection.cursor()
        cursor.execute(create_table_command)
        connection.commit()

In [94]:
create_news_table()

In [65]:
dataset = pd.read_csv('data/news_export_010224-23h33.csv')
rows = [tuple(row) for row in dataset.values]

In [86]:
dataset.columns.tolist()

['country', 'source_website', 'author', 'title', 'publishedAt']

In [98]:
def insert_news(rows):
    """ 
        Insert multiple news into the vendors table.
    """

    insert_statement = sql.SQL("INSERT INTO {} ({}) VALUES ({});").format(
        sql.Identifier('news'),
        sql.SQL(', ').join(map(sql.Identifier, 
                               ['country', 'source_website', 'author', 'title', 'published_at'])),
        sql.SQL(', ').join(sql.Placeholder() * 5)
    )
    with psycopg.connect(**POSTGRES_CONFIG) as connection:
        cursor = connection.cursor()
        for values in rows:
            cursor.execute(insert_statement, values)
        connection.commit()

In [99]:
insert_news(rows)